# **Sistemas Multi-Agente y ChromaDB con LlamaIndex** 🤖🗄️

Este notebook forma parte del curso de **IA Generativa** de la [Fundación GoodJob](https://www.fundaciongoodjob.org/).  
En él, los alumnos aprenderán a crear **sistemas multi-agente** avanzados que pueden trabajar con **bases de datos vectoriales** y **múltiples fuentes de información**.  

<p align="center">
  <img src="../notebooks/sources/llamaindex.png" alt="LlamaIndex" style="width:100%; height:auto;"/>
</p>


## ¿Qué vamos a aprender hoy? 🎯

En el notebook anterior aprendimos a crear agentes básicos con herramientas simples. ¡Ahora vamos a dar un salto a agentes más complejos! 

Hoy construiremos:

- 🗄️ **Bases de datos vectoriales** con ChromaDB para almacenar conocimiento
- 👥 **Sistemas multi-agente** con especialistas para diferentes tareas
- 🧠 **Agentes inteligentes** que pueden consultar documentos, APIs y hacer cálculos
- 🎯 **Routers** que deciden qué agente usar para cada consulta
- 🔄 **Orquestación** para combinar múltiples fuentes de información

¡Empezamos esta aventura! 🌟


## Prerrequisitos y configuración inicial 🔧

Antes de empezar, necesitamos instalar algunas dependencias adicionales para trabajar con bases de datos vectoriales y sistemas multi-agente.


In [3]:
# Install additional dependencies if needed
# Check if we have everything we need
import os
import sys
from pathlib import Path

print("🔍 Verificando configuración del entorno...")

# Check if data directory exists
data_dir = Path("../data")
if not data_dir.exists():
    print("📁 Creando directorio de datos...")
    data_dir.mkdir(exist_ok=True)
    (data_dir / "pdfs").mkdir(exist_ok=True)
    print("✅ Directorios creados: ../data/pdfs")
else:
    print("✅ Directorio de datos encontrado")

# Check if ChromaDB directory exists
chroma_dir = Path("../chroma_db")
if not chroma_dir.exists():
    print("🗄️ Preparando directorio para ChromaDB...")
    chroma_dir.mkdir(exist_ok=True)
    print("✅ Directorio ChromaDB creado")
else:
    print("✅ Directorio ChromaDB encontrado")

print("🎉 ¡Configuración inicial completada!")


🔍 Verificando configuración del entorno...
✅ Directorio de datos encontrado
✅ Directorio ChromaDB encontrado
🎉 ¡Configuración inicial completada!


## 1. ¿Qué es una base de datos vectorial? 🧠

### El problema que resuelven

Imagina que tienes **miles de documentos PDF** con información importante de tu empresa. Un usuario te pregunta: *"¿Cuál es la política de reembolsos para clientes internacionales?"*

Con métodos tradicionales tendrías que:
1. 😰 Buscar manualmente en cada documento
2. 🔍 Usar búsqueda por palabras clave (que puede fallar si usan sinónimos)
3. ⏰ Invertir horas en encontrar la respuesta

### La solución: Bases de datos vectoriales

Una **base de datos vectorial** convierte tus textos en **vectores numéricos** (embeddings) que capturan el **significado semántico**. Esto permite:

- 🎯 **Búsqueda semántica**: Encuentra contenido por significado, no solo palabras exactas
- ⚡ **Velocidad**: Consultas en milisegundos, incluso con millones de documentos  
- 🧠 **Inteligencia**: Entiende sinónimos, contexto y relaciones conceptuales
- 💾 **Persistencia**: Guarda todo para reutilizar sin reprocesar

### ¿Por qué ChromaDB?

**ChromaDB** es perfecto para aprender y proyectos porque es:
- 🚀 **Fácil de instalar**: Un simple `pip install`
- 💾 **Persistente**: Guarda tus datos localmente
- 🔧 **Integrado**: Funciona perfectamente con LlamaIndex
- 🆓 **Gratuito**: Sin costos de infraestructura

```
📄 PDF → 📝 Chunks → 🧮 Embeddings → 🗄️ ChromaDB → 🔍 Consulta → 🤖 LLM → ✨ Respuesta
```

Aunque en este curso utilizaremos **ChromaDB** por su simplicidad y porque requiere pocos recursos, existen muchas otras opciones muy válidas y potentes para bases de datos vectoriales. Algunas alternativas populares incluyen:
 
 - [Qdrant](https://qdrant.tech/): Motor de base de datos vectorial de alto rendimiento, open source y con muchas integraciones.
 - [Weaviate](https://weaviate.io/): Plataforma de base de datos vectorial con capacidades de búsqueda semántica y módulos de IA integrados.
 - [Pinecone](https://www.pinecone.io/): Servicio gestionado en la nube, muy utilizado en producción.
 - [Milvus](https://milvus.io/): Sistema open source escalable para gestión de vectores.
 - [PostgreSQL + pgvector](https://github.com/pgvector/pgvector): Extensión para PostgreSQL que permite almacenar y consultar vectores en una base de datos relacional tradicional.
 
Todas estas opciones son ampliamente utilizadas en la industria y pueden ser más adecuadas para proyectos en producción o con necesidades específicas.



## 1.1 ¿Qué es Top-K en Búsqueda Vectorial? 🎯

El **Top-K** es un concepto fundamental en las bases de datos vectoriales que determina **cuántos documentos similares** devolver para cada consulta.

### 🔍 ¿Cómo funciona Top-K?

Cuando haces una consulta en una base de datos vectorial:

1. **📝 Tu consulta** se convierte en un vector (embedding)
2. **📊 Se calcula la similitud** entre tu vector y todos los vectores almacenados
3. **🏆 Se ordenan por similitud** (más similar = menor distancia)
4. **🎯 Se devuelven los K más similares** (donde K es el número que configures)

<p align="center">
  <img src="sources/tok_k.jpg" alt="Funcionamiento Top-K en Base de Datos Vectorial" style="width:80%; height:auto;"/>
</p>

**Flujo de datos:**
- Los **Text Chunks** (documentos fragmentados) se procesan con **Encoders**
- Cada chunk genera su **Embedding** correspondiente
- Todos los embeddings se almacenan en la **Vector DB**
- Al consultar, se comparan similitudes y se devuelven los **Top-K más relevantes**

### ⚙️ Configurando Top-K: ¿Qué valor usar?

| **Top-K** | **Uso Recomendado** | **Ventajas** | **Desventajas** |
|-----------|-------------------|-------------|----------------|
| **1-3** | Respuestas muy específicas | 🎯 Máxima precisión, respuestas concisas | ⚠️ Puede perder contexto importante |
| **3-5** | Uso general (Recomendado) | ⚖️ Buen balance precisión/contexto | ✅ Ideal para la mayoría de casos |
| **5-8** | Análisis profundo | 📚 Contexto rico, análisis completo | 🐌 Respuestas más largas |
| **8+** | Investigación exhaustiva | 🔍 Contexto máximo | ⚠️ Riesgo de información irrelevante |

### 💡 Factores que Influyen en Top-K

1. **📏 Longitud del contexto del LLM**: Modelos con más contexto pueden procesar más chunks
2. **🎯 Tipo de consulta**: Preguntas específicas necesitan menos chunks
3. **📊 Calidad de los embeddings**: Mejores embeddings = mejor precisión con menos chunks
4. **⚡ Rendimiento**: Más chunks = más tiempo de procesamiento



## 1.2 ¿Cómo se Calcula la Similitud entre Embeddings? 📐

Para que una base de datos vectorial funcione, necesita una forma de **medir qué tan similares** son dos vectores. La métrica más común y efectiva es la **similitud del coseno**.

### 🧮 Similitud del Coseno: La Matemática Detrás de la Magia

La similitud del coseno mide el **ángulo entre dos vectores**, no su magnitud. Esto es perfecto para embeddings porque nos interesa la **dirección** (significado) más que la "intensidad".

<p align="center">
  <img src="sources/cosine_similarity.png" alt="Cálculo de Similitud del Coseno" style="width:85%; height:auto;"/>
</p>


#### **Fórmula Matemática:**

$
\cos(\theta) = \frac{A \cdot B}{\|A\| \, \|B\|}
$


Donde:
- **A · B**: Producto de los vectores
- **||A||**: Magnitud (norma) del vector A  
- **||B||**: Magnitud (norma) del vector B
- **θ**: Ángulo entre los vectores

### 🎯 ¿Por qué Coseno y no Distancia Euclidiana?

#### ✅ **Similitud del Coseno (Recomendada)**
- 🎯 **Insensible a magnitud**: Solo importa la dirección
- 📝 **Ideal para texto**: Captura significado semántico
- 🔢 **Rango normalizado**: Siempre entre 0 y 1
- ⚡ **Eficiente**: Cálculo optimizado

#### ❌ **Distancia Euclidiana (Menos ideal para texto)**
- 📏 **Sensible a magnitud**: Vectores largos parecen más diferentes
- 🔍 **Mejor para coordenadas**: Ideal para datos espaciales
- 📊 **Rango variable**: No está normalizada



## 1.3 Tipos de conexiones de bases de datos vectoriales con ChromaDB

ChromaDB ofrece múltiples formas de conectarse y almacenar datos, cada una optimizada para diferentes casos de uso. Vamos a explorar las tres opciones principales:

### 🔧 Opción 1: ChromaDB Persistente (Recomendado para desarrollo)

Esta es la opción que usaremos principalmente en el curso. Los datos se guardan en disco y persisten entre sesiones:

**Ventajas:**
- 💾 **Persistencia**: Los datos se mantienen entre sesiones
- 🔄 **Reutilización**: No necesitas reprocesar documentos
- 🏠 **Local**: Todo funciona sin conexión a internet
- 📁 **Organización**: Fácil backup y gestión de datos

**Cuándo usar:**
- Desarrollo y experimentación
- Proyectos personales
- Cuando necesitas persistir datos localmente

In [5]:
# OPCIÓN 1: ChromaDB Persistente - Los datos se guardan en disco
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

# Crear cliente persistente (los datos se guardan en ../chroma_db)
client = chromadb.PersistentClient(path="../chroma_db")
collection = client.get_or_create_collection("company_policies_persistent")

print("✅ ChromaDB Persistente configurado")
print("💾 Los datos se guardarán en: ../chroma_db")
print("🔄 Los datos persistirán entre sesiones")


✅ ChromaDB Persistente configurado
💾 Los datos se guardarán en: ../chroma_db
🔄 Los datos persistirán entre sesiones


### ⚡ Opción 2: ChromaDB en Memoria (Para experimentos rápidos)

Ideal para testing y experimentos donde no necesitas persistir los datos:

**Ventajas:**
- ⚡ **Velocidad máxima**: Todo en RAM
- 🧪 **Testing**: Perfecto para pruebas rápidas
- 🧹 **Limpieza automática**: No deja archivos residuales
- 🔄 **Reinicio limpio**: Cada ejecución empieza desde cero

**Cuándo usar:**
- Pruebas y experimentos temporales
- Desarrollo de prototipos
- Cuando no necesitas persistir datos
- Testing automatizado


In [6]:
# OPCIÓN 2: ChromaDB en Memoria - Máxima velocidad, datos temporales
import chromadb

# Crear cliente en memoria (los datos se pierden al reiniciar)
client_memory = chromadb.Client()  # Sin path = en memoria
collection_memory = client_memory.get_or_create_collection("temp_policies")

print("✅ ChromaDB en Memoria configurado")
print("⚡ Máxima velocidad de acceso")
print("⚠️ Los datos se perderán al reiniciar el programa")

# Ejemplo de uso rápido
collection_memory.add(
    documents=["Documento de prueba en memoria"],
    ids=["test_1"]
)

# Se usará un modelo de embeddings local para ahorrar costes
results = collection_memory.query(
    query_texts=["documento"],
    n_results=1
)
print(f"🧪 Test exitoso: {len(results['documents'][0])} documento encontrado")


✅ ChromaDB en Memoria configurado
⚡ Máxima velocidad de acceso
⚠️ Los datos se perderán al reiniciar el programa
🧪 Test exitoso: 1 documento encontrado


### 🐳 Opción 3: ChromaDB con Docker (Para producción)

Docker permite ejecutar ChromaDB como un servicio independiente, ideal para producción y equipos:

**¿Por qué Docker?**
- 🔒 **Aislamiento**: La base de datos corre en su propio contenedor
- 🚀 **Escalabilidad**: Fácil de escalar y gestionar recursos
- 👥 **Colaboración**: Mismo entorno para todo el equipo
- 🔄 **CI/CD**: Integración perfecta con pipelines de despliegue
- 🌐 **Acceso remoto**: Múltiples clientes pueden conectarse

**Comandos Docker para ChromaDB:**

```bash
# 1. Ejecutar ChromaDB en Docker
docker run -p 8000:8000 chromadb/chroma

# 2. Con persistencia de datos
docker run -p 8000:8000 -v ./chroma_data:/chroma/chroma chromadb/chroma

# 3. Con docker-compose (recomendado para producción)
# Crear archivo docker-compose.yml con configuración completa
```

**Conexión Python**


In [7]:
# OPCIÓN 3: ChromaDB con Docker - Para producción y equipos
import chromadb

# Conectar a ChromaDB ejecutándose en Docker
# Nota: Primero debes ejecutar: docker run -p 8000:8000 chromadb/chroma

try:
    # Conectar al servidor ChromaDB en Docker
    client_docker = chromadb.HttpClient(host='localhost', port=8000)
    collection_docker = client_docker.get_or_create_collection("production_policies")
    
    print("✅ ChromaDB Docker configurado")
    print("🐳 Conectado al contenedor en localhost:8000")
    print("🏢 Listo para producción")
    
except Exception as e:
    print("⚠️ ChromaDB Docker no disponible")
    print("💡 Para usar esta opción, ejecuta: docker run -p 8000:8000 chromadb/chroma")
    print(f"📝 Error: {e}")

# Ejemplo de docker-compose.yml para producción:
docker_compose_example = """
version: '3.8'
services:
  chromadb:
    image: chromadb/chroma:latest
    ports:
      - "8000:8000"
    volumes:
      - ./chroma_data:/chroma/chroma
    environment:
      - CHROMA_SERVER_HOST=0.0.0.0
      - CHROMA_SERVER_PORT=8000
"""

print("📋 Ejemplo de docker-compose.yml:")
print(docker_compose_example)


⚠️ ChromaDB Docker no disponible
💡 Para usar esta opción, ejecuta: docker run -p 8000:8000 chromadb/chroma
📝 Error: Could not connect to a Chroma server. Are you sure it is running?
📋 Ejemplo de docker-compose.yml:

version: '3.8'
services:
  chromadb:
    image: chromadb/chroma:latest
    ports:
      - "8000:8000"
    volumes:
      - ./chroma_data:/chroma/chroma
    environment:
      - CHROMA_SERVER_HOST=0.0.0.0
      - CHROMA_SERVER_PORT=8000



> ### 💡 Consejos Prácticos  
> Para desarrollo local usa Persistente (cambia a Memoria en tests rápidos), colabora con Docker para compartir configuración, recuerda que solo Persistente y Docker mantienen datos entre reinicios, y ten en cuenta que en rendimiento: Memoria > Docker > Persistente.  




## 2. Configurando nuestro LLM y embeddings 🔧

Primero, vamos a configurar nuestro modelo de lenguaje y el modelo de embeddings que convertirá nuestros textos en vectores.


In [ ]:
from huggingface_hub import InferenceClient

from sklearn.metrics.pairwise import cosine_similarity


def get_embedding(text: str) -> list[float]:
    client = InferenceClient(
    provider="nebius"
    )

    result = client.feature_extraction(
        text,
        model="Qwen/Qwen3-Embedding-8B",
    )
    return result[0]

king_embedding = get_embedding("King")
queen_embedding = get_embedding("Queen")
cat_embedding = get_embedding("Cat")

print(cosine_similarity([king_embedding], [queen_embedding]))
print(cosine_similarity([king_embedding], [cat_embedding]))



[[0.7313868]]
[[0.5848896]]


> ⚠️ Nota para el curso: Este bloque solo debe ejecutarse si tienes créditos en Hugging Face o si deseas usar un modelo local.


In [ ]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.embeddings.base_embedding import HuggingFaceInferenceEmbedding
from dotenv import load_dotenv


# Configure embeddings - using a LOCAL open model . Requiere GPU!!
model_name = "Qwen/Qwen3-Embedding-8B"
# Settings.embed_model = HuggingFaceEmbedding(
#     model_name= model_name
# )
Settings.embed_model = HuggingFaceInferenceEmbedding(
    model=model_name,
    provider="nebius"
)

model_name = "text-embedding-ada-002"
Settings.embed_model = OpenAIEmbedding(embed_batch_size=10)

print(f"✅ Modelo de embeddings configurado: {model_name}")
print("🎉 ¡Configuración completada!")

✅ Modelo de embeddings configurado: text-embedding-3-small
🎉 ¡Configuración completada!


<div align="center">
  <iframe src="https://projector.tensorflow.org/" width="1000" height="600"></iframe>
</div>


## 3. Creando nuestra primera base de datos vectorial 🗄️

Ahora vamos a crear una base de datos vectorial con ChromaDB. Para este ejemplo, vamos a crear algunos documentos de prueba que simulen políticas empresariales.
Más adelante en el curso profundizaremos en el parseo seguro de documentos utilizando `Pydantic` para garantizar la validez y estructura de los datos.


In [4]:
from llama_index.core import Document, VectorStoreIndex, StorageContext
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

# Create sample documents (in a real scenario, you'd load PDFs)
sample_docs = [
    Document(text="""
    POLÍTICA DE TRANSFERENCIAS INTERNACIONALES
    
    Las transferencias SWIFT a países de la Unión Europea tienen una comisión fija de 15 EUR 
    más el 0.2% del importe transferido. Para transferencias a Estados Unidos, la comisión 
    es de 25 USD más el 0.3% del importe. Las transferencias SEPA dentro de la eurozona 
    son gratuitas para importes superiores a 1000 EUR, y tienen una comisión de 2 EUR 
    para importes inferiores.
    """),
    
    Document(text="""
    POLÍTICA DE REEMBOLSOS Y DEVOLUCIONES
    
    Los reembolsos se procesan en un plazo máximo de 5 días laborables para clientes 
    nacionales y 10 días laborables para clientes internacionales. Se aplica una 
    comisión de gestión del 1% sobre el importe reembolsado, con un mínimo de 5 EUR. 
    Los reembolsos superiores a 10,000 EUR requieren aprobación del departamento de 
    riesgo y pueden tardar hasta 15 días laborables.
    """),
    
    Document(text="""
    TIPOS DE CAMBIO Y CONVERSIONES
    
    Los tipos de cambio se actualizan cada hora durante el horario comercial. 
    Para conversiones EUR/USD se aplica un spread del 0.5%, para EUR/GBP del 0.3% 
    y para otras divisas del 0.8%. Las conversiones automáticas se realizan al 
    tipo de cambio vigente en el momento de la transacción más el spread correspondiente.
    """)
]

print(f"📄 Documentos de ejemplo creados: {len(sample_docs)}")

# Parse documents into chunks
parser = SentenceSplitter(chunk_size=500, chunk_overlap=50)
nodes = parser.get_nodes_from_documents(sample_docs)
print(f"📝 Chunks generados: {len(nodes)}")

# Create ChromaDB client and collection
client = chromadb.PersistentClient(path="../chroma_db")
collection = client.get_or_create_collection("company_policies")

# Create vector store and index
vector_store = ChromaVectorStore(chroma_collection=collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create the index (this will generate embeddings and store them)
print("🧮 Generando embeddings y almacenando en ChromaDB...")
index_docs = VectorStoreIndex(nodes, storage_context=storage_context)

# Create query engine
qe_docs = index_docs.as_query_engine(similarity_top_k=3)

print("✅ ¡Base de datos vectorial creada y lista para consultas!")
print("💾 Los datos están persistidos en ../chroma_db")


📄 Documentos de ejemplo creados: 3
📝 Chunks generados: 3
🧮 Generando embeddings y almacenando en ChromaDB...
✅ ¡Base de datos vectorial creada y lista para consultas!
💾 Los datos están persistidos en ../chroma_db


### ¡Probemos nuestra base de datos vectorial! 🔍

Ahora vamos a hacer algunas consultas para ver cómo funciona la búsqueda semántica:


In [5]:
# Test our vector database with semantic search
print("🔍 Probando búsqueda semántica en nuestra base de datos vectorial...\n")

# Test queries
test_queries = [
    "¿Cuánto cuesta enviar dinero a Estados Unidos?",
    "¿Cuál es el tiempo de procesamiento para devoluciones internacionales?",
    "¿Qué spread se aplica para cambiar euros a libras?"
]

for i, query in enumerate(test_queries, 1):
    print(f"📝 Consulta {i}: {query}")
    response = qe_docs.query(query)
    print(f"🤖 Respuesta: {response.response}")
    print("-" * 80)

print("✨ ¡Increíble! La base de datos vectorial entiende el significado de nuestras preguntas")


🔍 Probando búsqueda semántica en nuestra base de datos vectorial...

📝 Consulta 1: ¿Cuánto cuesta enviar dinero a Estados Unidos?
🤖 Respuesta: Sending money to the United States costs 25 USD plus 0.3% of the transfer amount.
--------------------------------------------------------------------------------
📝 Consulta 2: ¿Cuál es el tiempo de procesamiento para devoluciones internacionales?
🤖 Respuesta: El tiempo de procesamiento para devoluciones internacionales es de 10 días laborables.
--------------------------------------------------------------------------------
📝 Consulta 3: ¿Qué spread se aplica para cambiar euros a libras?
🤖 Respuesta: El spread del 0.3% se aplica para cambiar euros a libras.
--------------------------------------------------------------------------------
✨ ¡Increíble! La base de datos vectorial entiende el significado de nuestras preguntas



Según la documentación oficial, el `ResponseSynthesizer` es el componente que genera la respuesta final usando un LLM, a partir de los fragmentos recuperados y la consulta del usuario.

Si no defines un LLM explícitamente ni un response_synthesizer, LlamaIndex utiliza los valores globales por defecto de Settings:
   - El LLM por defecto es Settings.llm (si no lo configuras, llama-index impondrá un modelo básico).
   - El modo de síntesis de respuesta por defecto es 'compact', aunque hay otros modos distintos que puedes explorar (ver en la [documentación](https://docs.llamaindex.ai/en/stable/module_guides/querying/response_synthesizers/")).
 


## 4 Sistemas Multi-agente 🤖
Los sistemas multiagente en LlamaIndex permiten coordinar múltiples agentes inteligentes, cada uno especializado en tareas o dominios específicos, para resolver problemas complejos de manera colaborativa. Estos sistemas combinan modelos de lenguaje, herramientas y flujos de trabajo personalizados, facilitando la creación de soluciones avanzadas y adaptables en entornos empresariales y de investigación.

### 4.1 Fundamentos de Agentes en LlamaIndex 

Antes de crear nuestro sistema multi-agente, es importante entender los **tipos de agentes** disponibles en LlamaIndex y cuándo usar cada uno.

#### 🎭 Tipos de Agentes en LlamaIndex

##### 1. **AgentWorkflow** (Recomendado - Más flexible)
- ✅ **Ventajas**: Máxima flexibilidad, control total del flujo
- ✅ **Uso**: Sistemas complejos, múltiples agentes, workflows personalizados
- 🎯 **Ideal para**: Nuestro sistema multi-agente

##### 2. **ReActAgent** (Clásico - Razonamiento explícito)  
- ✅ **Ventajas**: Razonamiento paso a paso visible, debugging fácil
- ✅ **Uso**: Cuando necesitas ver el proceso de razonamiento
- 🎯 **Ideal para**: Tareas que requieren explicabilidad

##### 3. **FunctionAgent** (Especializado - Para LLMs con function calling)
- ✅ **Ventajas**: Optimizado para modelos con function calling nativo
- ✅ **Uso**: Con modelos como GPT-4, Claude, etc.
- 🎯 **Ideal para**: Máximo rendimiento con modelos compatibles

#### 🧠 Anatomía de un Agente

Todos los agentes en LlamaIndex tienen estos componentes clave:

```
🤖 AGENTE = 🧠 LLM + 🛠️ Tools + 📋 System Prompt + 🔄 Workflow Logic
```

- **🧠 LLM**: El "cerebro" que toma decisiones
- **🛠️ Tools**: Las "manos" que ejecutan acciones  
- **📋 System Prompt**: Las "instrucciones" de comportamiento
- **🔄 Workflow Logic**: El "proceso" de razonamiento

#### 🎯 Agentes Especializados vs Generales

**🎯 Agentes Especializados** (Recomendado):
- Un agente por dominio específico
- Mayor precisión en su área
- Más fácil de debuggear y optimizar

**🌐 Agentes Generales**:
- Un agente con todas las herramientas
- Más simple de configurar
- Puede confundirse con muchas opciones


## 4. Creando nuestros agentes especialistas 👥

Ahora viene la parte emocionante: vamos a crear un equipo de agentes especialistas. Cada uno será experto en un área específica:

- 📚 **DocsAgent**: Especialista en consultar documentos
- 🌐 **APIAgent**: Experto en herramientas externas (clima, finanzas)  
- 🧮 **MathAgent**: Genio de los cálculos matemáticos

### Primero, creemos las herramientas para cada especialista


In [ ]:
from llama_index.core.tools import FunctionTool, QueryEngineTool
from llama_index.core.agent import ReActAgent
import random
import datetime as dt

# 1. DOCS AGENT TOOLS - Wrapping our query engine as a tool
policy_tool = QueryEngineTool.from_defaults(
    query_engine=qe_docs,
    name="policies_search",
    description="Busca información en las políticas y documentos de la empresa sobre transferencias, reembolsos, tipos de cambio y comisiones."
)

print("📚 Herramienta de documentos creada")

# 2. API AGENT TOOLS - Simulated external APIs
def fx_lookup(base: str, quote: str) -> dict:
    """Consulta tipos de cambio entre divisas"""
    # Mock exchange rates for demo
    mock_rates = {
        ("EUR", "USD"): 1.08, 
        ("USD", "EUR"): 0.93,
        ("EUR", "GBP"): 0.84, 
        ("GBP", "EUR"): 1.19,
        ("USD", "JPY"): 156.1,
        ("JPY", "USD"): 0.0064
    }
    
    rate = mock_rates.get((base.upper(), quote.upper()))
    if rate is None:
        return {"error": f"Par de divisas {base}/{quote} no soportado"}
    
    return {
        "base": base.upper(), 
        "quote": quote.upper(), 
        "rate": rate, 
        "timestamp": dt.datetime.utcnow().isoformat()
    }

def current_weather(city: str) -> dict:
    """Consulta el clima actual de una ciudad"""
    # Mock weather data for demo
    temp = round(18 + random.random() * 12, 1)  # Temperature between 18-30°C
    conditions = ["soleado", "nublado", "lluvia ligera", "despejado"]
    
    return {
        "city": city,
        "temperature_C": temp,
        "condition": random.choice(conditions),
        "timestamp": dt.datetime.utcnow().isoformat()
    }

# Create API tools
fx_tool = FunctionTool.from_defaults(
    fn=fx_lookup, 
    name="fx_lookup",
    description="Consulta tipos de cambio actuales entre dos divisas. Parámetros: base (divisa origen), quote (divisa destino)"
)

weather_tool = FunctionTool.from_defaults(
    fn=current_weather, 
    name="current_weather",
    description="Consulta el clima actual de una ciudad específica"
)

print("🌐 Herramientas de APIs externas creadas")

# 3. MATH AGENT TOOLS - Safe calculator
def safe_calculator(expression: str) -> dict:
    """Calculadora segura para operaciones matemáticas"""
    import re
    
    # Only allow numbers, basic operators, and parentheses
    if not re.fullmatch(r"[0-9\.\+\-\*\/\(\) ]+", expression):
        return {"error": "Expresión no permitida. Solo se permiten números y operadores básicos (+, -, *, /, ())"}
    
    try:
        result = eval(expression)
        return {
            "expression": expression,
            "result": result,
            "timestamp": dt.datetime.utcnow().isoformat()
        }
    except Exception as e:
        return {"error": f"Error en el cálculo: {str(e)}"}

calc_tool = FunctionTool.from_defaults(
    fn=safe_calculator, 
    name="calculator",
    description="Realiza cálculos matemáticos seguros. Acepta expresiones con números y operadores básicos (+, -, *, /, ())"
)

print("🧮 Herramienta de cálculo creada")
print("✅ ¡Todas las herramientas especializadas están listas!")


### Ahora creemos nuestros agentes especialistas 🎭


In [ ]:
# Create specialized agents
print("👥 Creando equipo de agentes especialistas...\n")

# 📚 DOCS AGENT - Expert in company documents
docs_agent = ReActAgent.from_tools(
    [policy_tool], 
    llm=Settings.llm, 
    max_iterations=4, 
    verbose=True
)
print("📚 DocsAgent creado - Especialista en documentos empresariales")

# 🌐 API AGENT - Expert in external data
api_agent = ReActAgent.from_tools(
    [fx_tool, weather_tool], 
    llm=Settings.llm, 
    max_iterations=4, 
    verbose=True
)
print("🌐 APIAgent creado - Especialista en datos externos")

# 🧮 MATH AGENT - Expert in calculations
math_agent = ReActAgent.from_tools(
    [calc_tool], 
    llm=Settings.llm, 
    max_iterations=3, 
    verbose=True
)
print("🧮 MathAgent creado - Especialista en cálculos")

# 🎭 GENERAL AGENT - Fallback with all tools
general_agent = ReActAgent.from_tools(
    [policy_tool, fx_tool, weather_tool, calc_tool],
    llm=Settings.llm, 
    max_iterations=5, 
    verbose=True
)
print("🎭 GeneralAgent creado - Agente general con todas las herramientas")

print("\n✅ ¡Equipo de agentes especialistas completado!")
print("🎉 Tenemos 4 agentes listos para trabajar en equipo")


## 5. El Router Inteligente: El Director del Equipo 🎯

Ahora necesitamos un **router inteligente** que decida qué agente debe manejar cada consulta. Vamos a crear un sistema híbrido que combina:

1. **Reglas simples** para casos obvios (rápido y eficiente)
2. **Clasificación con LLM** para casos ambiguos (inteligente y flexible)

### ¿Cómo funciona el router?

```
Consulta → Reglas simples → ¿Claro? → Sí → Agente especialista
                        ↓
                       No → LLM Router → Agente especialista
```


In [ ]:
def route_with_rules(message: str) -> str:
    """
    Router basado en reglas simples para casos obvios
    Retorna: 'docs', 'api', 'math', o 'unknown'
    """
    msg_lower = message.lower()
    
    # Keywords for document-related queries
    docs_keywords = ["documento", "política", "pdf", "comisión", "transferencia", 
                     "reembolso", "según el documento", "política de", "apartado"]
    
    # Keywords for API-related queries  
    api_keywords = ["cambio", "fx", "clima", "tiempo", "meteo", "temperatura",
                   "eur", "usd", "gbp", "jpy", "divisa", "moneda"]
    
    # Keywords for math-related queries
    math_keywords = ["calcula", "suma", "resta", "multiplica", "divide", 
                     "porcentaje", "%", "resultado de", "cuánto es"]
    
    # Check for matches
    if any(keyword in msg_lower for keyword in docs_keywords):
        return "docs"
    elif any(keyword in msg_lower for keyword in api_keywords):
        return "api"
    elif any(keyword in msg_lower for keyword in math_keywords):
        return "math"
    else:
        return "unknown"

def route_with_llm(message: str) -> str:
    """
    Router inteligente usando LLM para casos ambiguos
    """
    router_prompt = f"""Eres un router inteligente. Analiza el siguiente mensaje y decide qué tipo de agente debe manejarlo.

Responde SOLO con una de estas palabras:
- 'docs' si necesita buscar en documentos internos de la empresa (políticas, comisiones, procedimientos)
- 'api' si necesita herramientas externas (tipos de cambio, clima, datos en tiempo real)
- 'math' si es principalmente un cálculo matemático
- 'general' para otros casos o consultas mixtas

Mensaje: {message}

Tu decisión:"""

    response = Settings.llm.complete(router_prompt)
    decision = response.text.strip().lower()
    
    # Validate response
    valid_decisions = {"docs", "api", "math", "general"}
    return decision if decision in valid_decisions else "general"

def smart_router(message: str) -> str:
    """
    Router híbrido: primero reglas simples, luego LLM si es necesario
    """
    # Try rule-based routing first
    rule_decision = route_with_rules(message)
    
    if rule_decision != "unknown":
        print(f"🎯 Router (reglas): {rule_decision}")
        return rule_decision
    else:
        # Fall back to LLM routing
        llm_decision = route_with_llm(message)
        print(f"🧠 Router (LLM): {llm_decision}")
        return llm_decision

def route_and_chat(message: str):
    """
    Función principal que enruta la consulta al agente apropiado
    """
    print(f"📝 Consulta: {message}")
    
    # Decide which agent to use
    agent_type = smart_router(message)
    
    # Route to appropriate agent
    if agent_type == "docs":
        print("📚 Enviando a DocsAgent...")
        return docs_agent.chat(message)
    elif agent_type == "api":
        print("🌐 Enviando a APIAgent...")
        return api_agent.chat(message)
    elif agent_type == "math":
        print("🧮 Enviando a MathAgent...")
        return math_agent.chat(message)
    else:
        print("🎭 Enviando a GeneralAgent...")
        return general_agent.chat(message)

print("🎯 ¡Router inteligente creado y listo para dirigir consultas!")


### ¡Probemos nuestro sistema multi-agente! 🚀

Ahora vamos a probar cómo nuestro router inteligente dirige las consultas a los agentes apropiados:


In [ ]:
# Test our multi-agent system
print("🎭 Probando nuestro sistema multi-agente...\n")

test_queries = [
    "¿Cuál es la comisión para transferencias SWIFT a Estados Unidos?",
    "¿A cuánto está el cambio EUR/USD hoy?", 
    "Calcula el 15% de 2500 euros",
    "¿Qué tiempo hace en Madrid?"
]

for i, query in enumerate(test_queries, 1):
    print(f"\n{'='*60}")
    print(f"🧪 PRUEBA {i}")
    print(f"{'='*60}")
    
    try:
        response = route_and_chat(query)
        print(f"✅ Respuesta: {response.response}")
    except Exception as e:
        print(f"❌ Error: {e}")
    
    print(f"{'='*60}")

print("\n🎉 ¡Sistema multi-agente funcionando perfectamente!")


## 6. Orquestación Multi-Fuente: Combinando Agentes 🎼

A veces una consulta requiere **múltiples especialistas** trabajando juntos. Por ejemplo: *"Si envío 1200 EUR por SWIFT a Estados Unidos, ¿cuánto llega después de comisiones y a qué tipo de cambio?"*

Esta consulta necesita:
1. 📚 **DocsAgent** → Buscar comisiones SWIFT
2. 🧮 **MathAgent** → Calcular importe después de comisiones  
3. 🌐 **APIAgent** → Consultar tipo de cambio EUR/USD

### Creemos un orquestador inteligente


## 🚀 Sistemas Multi-Agente Avanzados

Ahora que entendemos los fundamentos, vamos a explorar arquitecturas más sofisticadas:

### 🎭 Arquitecturas Multi-Agente

#### 1. **Arquitectura Jerárquica** (Recomendada)
```
    🎭 Router Agent (Director)
         /      |      \
    📚 Docs   🌐 API   🧮 Math
    Agent     Agent    Agent
```

#### 2. **Arquitectura Colaborativa**
```
📚 Docs ←→ 🌐 API ←→ 🧮 Math
    ↑         ↑         ↑
    └─────────┼─────────┘
              ↓
        🎯 Coordinator
```

#### 3. **Arquitectura Pipeline**
```
📚 Docs → 🧮 Math → 🌐 API → 📊 Results
```

### 💡 Casos de Uso Avanzados

1. **🏦 Análisis Financiero Completo**
   - DocsAgent: Busca políticas de inversión
   - APIAgent: Obtiene datos de mercado en tiempo real
   - MathAgent: Calcula riesgos y rendimientos
   - ReportAgent: Genera informes ejecutivos

2. **🏥 Diagnóstico Médico Asistido**
   - KnowledgeAgent: Consulta bases de datos médicas
   - SymptomAgent: Analiza síntomas del paciente
   - TestAgent: Sugiere pruebas diagnósticas
   - RecommendationAgent: Propone tratamientos

3. **🎓 Tutor Inteligente Personalizado**
   - AssessmentAgent: Evalúa nivel del estudiante
   - ContentAgent: Busca material educativo apropiado
   - ExerciseAgent: Genera ejercicios personalizados
   - FeedbackAgent: Proporciona retroalimentación


In [ ]:
def orchestrate_multi_agent(message: str):
    """
    Orquestador inteligente para consultas que requieren múltiples agentes
    """
    msg_lower = message.lower()
    
    # Detect complex queries that need multiple agents
    if ("swift" in msg_lower or "sepa" in msg_lower) and any(currency in msg_lower for currency in ["usd", "eur", "gbp"]):
        print("🎼 Detectada consulta compleja - Iniciando orquestación multi-agente")
        
        # Step 1: Get commission information from documents
        print("\n📚 PASO 1: Consultando comisiones en documentos...")
        docs_query = f"¿Cuáles son las comisiones para {message}?"
        docs_response = docs_agent.chat(docs_query)
        print(f"📋 Información de comisiones: {docs_response.response}")
        
        # Step 2: Extract amount and calculate after commissions
        print("\n🧮 PASO 2: Calculando importe después de comisiones...")
        # Simple extraction (in production, you'd use more sophisticated NLP)
        import re
        amount_match = re.search(r'(\d+(?:\.\d+)?)\s*eur', msg_lower)
        if amount_match:
            amount = float(amount_match.group(1))
            # Simplified calculation - in reality, you'd parse the commission info
            commission_calc = f"{amount} - ({amount} * 0.003) - 25"  # Approximate SWIFT commission
            math_response = math_agent.chat(f"Calcula: {commission_calc}")
            print(f"💰 Importe después de comisiones: {math_response.response}")
        
        # Step 3: Get exchange rate
        print("\n🌐 PASO 3: Consultando tipo de cambio...")
        if "usd" in msg_lower:
            fx_response = api_agent.chat("¿Cuál es el tipo de cambio EUR/USD actual?")
            print(f"💱 Tipo de cambio: {fx_response.response}")
        
        # Step 4: Combine all information
        print("\n📊 RESUMEN COMPLETO:")
        combined_response = f"""
        Análisis completo de tu transferencia:
        
        📋 Comisiones: {docs_response.response}
        💰 Cálculo: {math_response.response if 'math_response' in locals() else 'No calculado'}
        💱 Tipo de cambio: {fx_response.response if 'fx_response' in locals() else 'No consultado'}
        
        Esta información te permite conocer el costo total y el importe final de tu transferencia.
        """
        
        return combined_response
    
    else:
        # For simple queries, use regular routing
        return route_and_chat(message)

def smart_chat(message: str):
    """
    Función principal que decide entre routing simple u orquestación compleja
    """
    print(f"💬 Consulta recibida: {message}")
    
    # Check if it needs orchestration
    if any(keyword in message.lower() for keyword in ["swift", "sepa"]) and \
       any(currency in message.lower() for currency in ["eur", "usd", "gbp"]) and \
       any(amount_word in message.lower() for amount_word in ["envío", "transferir", "mandar"]):
        return orchestrate_multi_agent(message)
    else:
        return route_and_chat(message)

print("🎼 ¡Orquestador multi-agente creado y listo!")


### ¡Probemos la orquestación multi-agente! 🎭

Vamos a probar una consulta compleja que requiere múltiples agentes trabajando en secuencia:


In [ ]:
# Test complex orchestration
print("🎼 Probando orquestación multi-agente...\n")

complex_query = "Si envío 1200 EUR por SWIFT a Estados Unidos, ¿cuánto llega después de comisiones y a qué tipo de cambio?"

print("="*80)
print("🧪 PRUEBA DE ORQUESTACIÓN COMPLEJA")
print("="*80)

try:
    result = smart_chat(complex_query)
    print(f"\n✅ RESULTADO FINAL:\n{result}")
except Exception as e:
    print(f"❌ Error en orquestación: {e}")

print("="*80)
print("\n🎉 ¡Orquestación multi-agente completada!")


## 7. Persistencia: Reutilizando nuestra base de datos 💾

Una de las ventajas de ChromaDB es que **persiste** nuestros datos. Esto significa que la próxima vez que ejecutemos el notebook, no necesitamos volver a procesar todos los documentos.

### ¿Cómo reconectar a una base de datos existente?


In [ ]:
# How to reconnect to existing ChromaDB
def reconnect_to_existing_db():
    """
    Función para reconectar a una base de datos ChromaDB existente
    """
    print("🔄 Reconectando a base de datos existente...")
    
    # Connect to existing ChromaDB
    client = chromadb.PersistentClient(path="../chroma_db")
    collection = client.get_or_create_collection("company_policies")
    
    # Create vector store from existing collection
    vector_store = ChromaVectorStore(chroma_collection=collection)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    
    # Recreate index from existing vector store
    index_docs = VectorStoreIndex.from_vector_store(
        vector_store=vector_store, 
        storage_context=storage_context
    )
    
    # Create query engine
    qe_docs = index_docs.as_query_engine(similarity_top_k=3)
    
    print("✅ Reconexión exitosa - Base de datos lista para usar")
    return qe_docs

# Check if we have data in our collection
collection_count = collection.count()
print(f"📊 Documentos en la colección: {collection_count}")

if collection_count > 0:
    print("💾 ¡Perfecto! Ya tienes datos persistidos en ChromaDB")
    print("🔄 En futuras ejecuciones, puedes usar reconnect_to_existing_db()")
else:
    print("📝 La colección está vacía - necesitas cargar documentos primero")


## 8. Ejercicios Prácticos para Estudiantes 🎯

¡Ahora es tu turno de experimentar! Aquí tienes algunos ejercicios para profundizar en los conceptos aprendidos:

### Ejercicio 1: Mejorando el Router 🎯

**Objetivo**: Crear un sistema de medición de precisión del router

**Tareas**:
1. Crea un conjunto de 12 consultas etiquetadas (3 para cada tipo: docs, api, math, general)
2. Mide la precisión del `route_with_rules` vs `route_with_llm`
3. Implementa un sistema de confianza que use LLM solo cuando la confianza de las reglas sea baja

**Código inicial**:


In [ ]:
# EJERCICIO 1: Sistema de evaluación del router
test_dataset = [
    # Docs queries
    ("¿Cuál es la política de reembolsos?", "docs"),
    ("¿Qué comisión se aplica a transferencias SEPA?", "docs"),
    ("Según el documento, ¿cuál es el plazo de procesamiento?", "docs"),
    
    # API queries  
    ("¿Cuál es el tipo de cambio EUR/USD?", "api"),
    ("¿Qué tiempo hace en Barcelona?", "api"),
    ("¿A cuánto está el cambio de libras a euros?", "api"),
    
    # Math queries
    ("Calcula el 15% de 2500", "math"),
    ("¿Cuánto es 1200 multiplicado por 1.08?", "math"),
    ("¿Cuál es el resultado de (500 + 300) * 0.02?", "math"),
    
    # General queries
    ("¿Cómo estás hoy?", "general"),
    ("Explícame qué es la inteligencia artificial", "general"),
    ("¿Cuál es tu nombre?", "general")
]

def evaluate_router_accuracy():
    """
    TODO: Implementa esta función para medir la precisión del router
    
    Debe:
    1. Probar cada consulta con route_with_rules y route_with_llm
    2. Comparar con la etiqueta correcta
    3. Calcular y mostrar métricas de precisión
    4. Mostrar casos donde fallan
    """
    print("🎯 EJERCICIO 1 - Implementa la evaluación del router")
    print("💡 Pista: Usa test_dataset para probar ambos métodos de routing")
    pass

# Descomenta para probar tu implementación
# evaluate_router_accuracy()


### Ejercicio 2: Agente de Cotización de Envíos 💰

**Objetivo**: Crear un agente especializado que calcule el costo total de envíos internacionales

**Especificaciones**:
- Input: `{importe, via (SWIFT|SEPA), destino (EUR|USD|GBP)}`
- Proceso: DocsAgent → MathAgent → APIAgent
- Output: Desglose completo paso a paso

**Código inicial**:


In [ ]:
def create_shipping_quote_agent():
    """
    TODO: Implementa un agente especializado en cotizaciones de envío
    
    Debe:
    1. Extraer parámetros de la consulta (importe, vía, destino)
    2. Consultar comisiones en documentos
    3. Calcular costo total con comisiones
    4. Obtener tipo de cambio si es necesario
    5. Presentar desglose detallado
    
    Ejemplo de uso:
    quote_agent("Quiero enviar 1500 EUR por SWIFT a Estados Unidos")
    
    Salida esperada:
    📋 Importe original: 1500 EUR
    💰 Comisión SWIFT: 25 USD + 0.3% = 29.50 USD
    💱 Tipo de cambio EUR/USD: 1.08
    📊 Total a enviar: 1470.50 EUR
    💵 Importe que llega: ~1588.14 USD
    """
    print("💰 EJERCICIO 2 - Implementa el agente de cotización")
    print("💡 Pista: Combina los 3 agentes especializados en secuencia")
    pass

# Descomenta para probar tu implementación
# shipping_agent = create_shipping_quote_agent()
# shipping_agent("Quiero enviar 1500 EUR por SWIFT a Estados Unidos")


### Ejercicio 3: Sistema de Observabilidad 📊

**Objetivo**: Implementar logging y métricas para monitorear el rendimiento del sistema

**Tareas**:
1. Log cada interacción en formato JSONL
2. Medir latencia de cada agente
3. Contar uso de herramientas
4. Generar reporte de rendimiento

**Código inicial**:


In [ ]:
import json
import time
from datetime import datetime

class AgentObserver:
    """
    TODO: Implementa un sistema de observabilidad para los agentes
    
    Debe incluir:
    - Log de cada consulta con timestamp
    - Medición de latencia por agente
    - Conteo de uso de herramientas
    - Detección de errores
    - Generación de reportes
    """
    
    def __init__(self):
        self.logs = []
        self.metrics = {
            "total_queries": 0,
            "agent_usage": {"docs": 0, "api": 0, "math": 0, "general": 0},
            "tool_usage": {},
            "total_latency": 0,
            "errors": 0
        }
    
    def log_interaction(self, query, agent_type, response, latency, tools_used=None, error=None):
        """TODO: Implementa el logging de interacciones"""
        print("📊 EJERCICIO 3 - Implementa el sistema de observabilidad")
        print("💡 Pista: Guarda cada interacción con timestamp, latencia y herramientas usadas")
        pass
    
    def generate_report(self):
        """TODO: Genera un reporte de rendimiento del sistema"""
        pass
    
    def save_logs_to_file(self, filename="agent_logs.jsonl"):
        """TODO: Guarda los logs en formato JSONL"""
        pass

# Crear observador global
observer = AgentObserver()

# TODO: Modifica la función route_and_chat para usar el observador
def monitored_route_and_chat(message: str):
    """
    Versión monitoreada de route_and_chat que incluye observabilidad
    """
    print("📊 EJERCICIO 3 - Implementa el monitoreo en esta función")
    print("💡 Pista: Mide tiempo, captura errores y registra todo en el observer")
    pass

# Descomenta para probar tu implementación
# monitored_route_and_chat("¿Cuál es la comisión SWIFT?")
# observer.generate_report()


## 9. Recursos para Expandir tu Base de Datos 📚

Para practicar con documentos reales, aquí tienes algunas fuentes públicas donde puedes descargar PDFs:

### Fuentes Recomendadas:

1. **📜 EUR-Lex / Diario Oficial de la UE**
   - Documentos legales multilingües
   - Ideal para practicar con políticas y regulaciones

2. **🏛️ Oficina de Publicaciones de la UE (op.europa.eu)**
   - Informes, guías y estudios oficiales
   - Gran variedad temática

3. **📊 SEC EDGAR (Estados Unidos)**
   - Informes financieros 10-K/10-Q
   - Perfecto para análisis financiero

4. **🔬 arXiv**
   - Papers científicos en PDF
   - Excelente para contenido técnico

5. **🏥 PubMed Central (Open Access)**
   - Artículos biomédicos
   - Ideal para dominio médico

6. **🌍 UN/World Bank/OECD**
   - Informes y análisis internacionales
   - Datos económicos y sociales


## 10. FAQ y Mejores Prácticas 🤔

### Preguntas Frecuentes:

**🤔 ¿Por qué ChromaDB y no otra base de datos vectorial?**
- ✅ **Fácil instalación**: Solo `pip install chromadb`
- ✅ **Persistencia local**: No necesitas infraestructura externa
- ✅ **Integración perfecta**: Funciona nativamente con LlamaIndex
- ✅ **Ideal para aprendizaje**: Perfecto para POCs y desarrollo

**🎯 ¿Cuántos documentos similares debería recuperar (top-k)?**
- **3-5**: Para consultas específicas y respuestas concisas
- **5-8**: Para análisis más profundos
- **8+**: Para investigación exhaustiva (cuidado con el límite de contexto)

**🔄 ¿Cómo combino múltiples colecciones temáticas?**
- Crea un `QueryEngine` por cada colección/tema
- Exponlos como herramientas (`QueryEngineTool`)
- Deja que el router decida cuál usar

**⚠️ ¿Cómo evito alucinaciones en las herramientas?**
- ✅ **Descripciones claras** en cada herramienta
- ✅ **Validaciones robustas** en las funciones
- ✅ **Límites de iteraciones** en los agentes
- ✅ **Logging completo** de todas las llamadas

### Mejores Prácticas:

1. **🎯 Especialización**: Un agente por dominio específico
2. **🔍 Validación**: Siempre valida inputs y outputs
3. **📊 Monitoreo**: Implementa observabilidad desde el inicio
4. **💾 Persistencia**: Usa ChromaDB para evitar reprocesamiento
5. **🧪 Testing**: Crea datasets de prueba etiquetados


## 🔍 LlamaTrace: Observabilidad y Debugging Avanzado

**LlamaTrace** es el sistema de observabilidad integrado de LlamaIndex que nos permite **monitorear, debuggear y optimizar** nuestros agentes y aplicaciones de IA en tiempo real.

### ¿Por qué necesitamos LlamaTrace?

Cuando trabajamos con sistemas multi-agente complejos, necesitamos responder preguntas como:
- 🤔 **¿Por qué mi agente tomó esa decisión?**
- ⏱️ **¿Cuánto tiempo tarda cada herramienta?**
- 💰 **¿Cuánto me está costando cada consulta?**
- 🐛 **¿Dónde está fallando mi sistema?**
- 📊 **¿Qué agente es más eficiente?**

### 🌟 Ventajas de LlamaTrace

#### 1. **Trazabilidad Completa**
- 📋 **Seguimiento paso a paso** de cada decisión del agente
- 🔗 **Cadena completa** de razonamiento y herramientas
- 📝 **Logs detallados** de todas las interacciones

#### 2. **Métricas en Tiempo Real**
- ⏱️ **Latencia** de cada componente
- 💰 **Costos** por token y por consulta
- 📊 **Uso de recursos** y rendimiento
- 🎯 **Precisión** y calidad de respuestas

#### 3. **Debugging Visual**
- 🖥️ **Interfaz web** para explorar trazas
- 🌳 **Árbol de decisiones** visual
- 📈 **Gráficos** de rendimiento y uso

#### 4. **Integración Automática**
- 🔄 **Sin configuración adicional** en la mayoría de casos
- 🔌 **Compatible** con todos los componentes de LlamaIndex
- 📤 **Exportación** a herramientas externas (Weights & Biases, etc.)

### 🚀 Casos de Uso Principales

1. **🔧 Desarrollo y Debugging**
   - Identificar cuellos de botella
   - Optimizar prompts y herramientas
   - Detectar errores en la lógica

2. **📊 Monitoreo en Producción**
   - Alertas de rendimiento
   - Análisis de costos
   - Métricas de calidad

3. **🧪 Experimentación**
   - A/B testing de agentes
   - Comparación de modelos
   - Optimización de parámetros

### 💡 ¿Cuándo usar LlamaTrace?

- ✅ **Siempre en desarrollo** para entender el comportamiento
- ✅ **En producción** para monitoreo continuo  
- ✅ **Durante optimización** para medir mejoras
- ✅ **Para debugging** cuando algo no funciona como esperamos

En las siguientes secciones veremos cómo activar y usar LlamaTrace con nuestros agentes multi-fuente.


## ¡Felicidades! 🎉 Has creado un sistema multi-agente avanzado

### 🏆 ¿Qué has logrado hoy?

En este notebook has construido un sistema sofisticado:

1. **🗄️ Base de datos vectorial** con ChromaDB para búsqueda semántica
2. **👥 Sistema multi-agente** con especialistas en diferentes dominios
3. **🎯 Router inteligente** que dirige consultas al agente apropiado
4. **🎼 Orquestación** para combinar múltiples agentes en consultas complejas
5. **💾 Persistencia** para reutilizar datos sin reprocesamiento

### 🚀 ¿Por qué es esto revolucionario?

- **🧠 Inteligencia distribuida**: Cada agente es experto en su dominio
- **⚡ Eficiencia**: El router optimiza qué agente usar
- **🔄 Escalabilidad**: Fácil añadir nuevos agentes y herramientas
- **💾 Persistencia**: No reprocesas datos innecesariamente
- **🎯 Precisión**: Especialización mejora la calidad de respuestas

### 🛣️ ¿Qué sigue?

En los próximos notebooks aprenderás:

- ✅ ✨ **Introducción a LLMs, agentes y tools**
- ✅ 🤖 **Sistemas multi-agente y ChromaDB**
- ⬜ 📄 **Parseo avanzado de documentos**
- ⬜ 🏗️ **Validación con Pydantic**
- ⬜ 🚀 **Despliegue en producción**


¡Esto es solo el comienzo de lo que puedes construir con IA Generativa! 🌟

**¡Sigue experimentando y construyendo el futuro! 🚀**
